In [ ]:
import os
from database_tools.tools import BuildDatabase

repo_dir = '/media/cam/CAPSTONEDB/database_tools/'
os.chdir(repo_dir)

config = dict(
    low=0.5,
    high=8.0,
    sim1=0.7,
    sim2=0.9,
    df=0.2,
    snr_t=2,
    hr_diff=1/6,
    f0_low=0.667,
    f0_high=3.0,
    abp_min_bounds=[40, 100],  # this is largest possible range of valid bp values
    abp_max_bounds=[70, 190],  # ^
)

worker = BuildDatabase(
    output_dir='data-2022-10-04/',
    config=config,
    win_len=256,
    fs=125,
    samples_per_file=2500,
    samples_per_patient=600,
    max_samples=2500,
    data_dir='physionet.org/files/mimic3wdb/1.0/',
)

df = worker.run()

In [9]:
import os
import pandas as pd
from database_tools.tools import DataEvaluator

pd.options.display.max_rows = 1000

repo_dir = '/media/cam/CAPSTONEDB/database_tools/'
os.chdir(repo_dir)

df = pd.read_csv('data-2022-10-04/mimic3_stats.csv')
worker = DataEvaluator(stats=df)

figs = worker.run()

In [11]:
df.head(20)

,mrn,valid,time_similarity,spectral_similarity,ppg_snr,abp_snr,ppg_hr,abp_hr,abp_min,abp_max
0,3508039,False,NaN,NaN,NaN,NaN,109.863281,1252.441406,0.000000,0.000000
1,3508039,False,1.752278e-01,0.636204,NaN,NaN,106.201172,18.310547,0.000000,185.021467
2,3508039,False,1.603994e-17,0.060205,NaN,NaN,106.201172,3.662109,185.021467,185.021467
3,3508039,False,-1.835806e-17,0.076946,NaN,NaN,106.201172,3.662109,185.021467,185.021467
4,3508039,False,-6.332338e-18,0.067010,NaN,NaN,109.863281,3.662109,185.021467,185.021467
5,3508039,False,2.510144e-17,0.063564,NaN,NaN,109.863281,3.662109,185.021467,185.021467
6,3508039,False,6.943528e-02,0.649990,NaN,NaN,106.201172,18.310547,-14.995140,185.021467
7,3508039,False,4.726607e-02,0.504728,NaN,-2.168155,106.201172,435.791016,-14.995140,40.882515
8,3508039,False,1.169564e-01,0.426620,NaN,-5.105101,109.863281,21.972656,-14.995140,78.052879
9,3508039,False,1.415857e-01,0.471489,NaN,-5.530835,106.201172,435.791016,-14.995140,53.435450


In [1]:
import glob
import json
import numpy as np
import pandas as pd

ppg = []
abp = []

with open('../data-2022-10-04/mimic3/lines/mimic3_0000000.jsonlines', 'r') as f:
    for sample in f:
        sample = json.loads(sample)
        ppg.append(sample['ppg'])
        abp.append(sample['abp'])


ppg = np.array(ppg).flatten()
abp = np.array(abp).flatten()

In [15]:
from scipy import signal

x = ppg[0:256]

freqs, psd = signal.periodogram(x, 125, nfft=2048)
# f0 = freqs[np.argmax(psd)]
f0, f1, f2 = freqs[np.sort(np.argpartition(psd, -3)[-3:])]
freq_res = freqs[1] - freqs[0]

In [23]:
np.argpartition(psd, -3)[-3:]

array([32, 30, 31])

In [19]:
pd.DataFrame(dict(x=freqs, y=psd)).plot(x='x', y='y')

In [ ]:
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

fig = go.FigureWidget()
fig.add_scatter()
fig.update_layout(
    xaxis_title='Frequency',
    yaxis_title='Amplitude',
    width=1000,
    height=800,
)

i = 0
fig.data[0].y = ppg[i * 256:(i + 1) * 256]
fig

In [ ]:
# for f in $files; do mongoimport --db MIMICDB --file $f; done

# from pymongo import MongoClient

# uri = 'mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+1.6.0'
# client = MongoClient(uri)

# coll = client.MIMICDB.mimic3_0000067

# cursor = coll.find()
# for doc in cursor:
#     print(doc)